In [2]:
import pandas as pd
import sys

from NewsSentiment import TargetSentimentClassifier

/home/ahaque2/venv/py3_7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Download the dataset and place under dataset/ folder

news = pd.read_csv('dataset/news.csv')
news_tweets = pd.read_csv('dataset/news_tweets.csv')
# user_resp = pd.read_csv('results/user_response.csv')

In [18]:
news.shape, news_tweets.shape

((35886, 32), (24584, 49))

## Compuate target based sentiment

In [36]:
def get_sentiment_scores(news_tweets, ent):
    
    news_tweets_filtered = news_tweets[news_tweets.text.str.contains(ent)]
    proc_head = []
    senti = []
    j = 0
    tsc = TargetSentimentClassifier()
    for i, n in enumerate(news_tweets_filtered.text):
        # print(type(n))
        k = n.split(ent)
        # print(k)
        sn = dict()
        j+=1
        if(ent in n):
            # print(x)
            # proc_head.append(x)
            x = [k[0], ent, k[1]]
            sentiment = tsc.infer_from_text(k[0].strip('\n'), ent, k[1])
            # print(x, sentiment)
            # print(sentiment[0]['class_label'], sentiment[0]['class_prob'])
            sn[sentiment[0]['class_label']] = sentiment[0]['class_prob']
            sn[sentiment[1]['class_label']] = sentiment[1]['class_prob']
            sn[sentiment[2]['class_label']] = sentiment[2]['class_prob']
            senti.append(sn)
            print(j, end = "\r")
            # sys.exit()
        else:
            senti.append(None)
            
    pos_t_senti = [x['positive'] if x != None else None for x in senti]
    neg_t_senti = [x['negative'] if x != None else None for x in senti]
    neu_t_senti = [x['neutral'] if x != None else None for x in senti]
    
    news_tweets_filtered['biden_pos_new'] = pos_t_senti
    news_tweets_filtered['biden_neg_new'] = neg_t_senti
    news_tweets_filtered['biden_neu_new'] = neu_t_senti
            
    return news_tweets_filtered

In [37]:
news_tweets_filtered_B = get_sentiment_scores(news_tweets, 'Biden')
news_tweets_filtered_T = get_sentiment_scores(news_tweets, 'Trump')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/home/ahaque2/venv/py3_7/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ahaque2/venv/py3_7/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ahaque2/venv/py3_7/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [39]:
news_tweets_senti = pd.concat((news_tweets_filtered_B, news_tweets_filtered_T), axis = 0)

(35, (35, 49))

In [ ]:
news_tweets_filtered = news_tweets_filtered.drop_duplicates('tweet_id')

In [180]:
news_tweets_senti.to_csv('results/news_tweets.csv')